### Imports básicos


In [1]:
from copy import deepcopy
import pandas as pd
import numpy as np
import os


### Setando nome das colunas baseados em latitude e longitude


In [2]:
w_values = range(60,0,-2)
e_values = range(0,15,2)
n_values = range(30,0,-2)
s_values = range(0,20,2)

colunas = []
linhas = []

# LINHA
for value in w_values:
    column_name = str(value) + "W"
    linhas.append(column_name)
    
for value in e_values:
    column_name = str(value) + "E"
    linhas.append(column_name)

# COLUNA    
for value in n_values:
    column_name = str(value) + "N"
    colunas.append(column_name)
    
for value in s_values:
    column_name = str(value) + "S"
    colunas.append(column_name)

colunas_do_data_frame = []
for linha in linhas:
    for coluna in colunas:
        lat_long = linha + "-" + coluna
        colunas_do_data_frame.append(lat_long)

### Lendo arquivo e substituindo as quebras de linhas por blank

In [3]:
file_name = "funceme_db/anomalia/geral/_Dados_TSMvento_2014_04_anomt6414b04"
file_content = open(file_name).read()
file_content = file_content.replace("\n", "")


### Constantes

In [4]:
# Todo mês possui 25 linhas por 38 colunas que dá 950
BLOCO_DE_DADOS_POR_MES = 950
BLOCO_TOTAL = 573800 #(950 blocos x 604 meses)

### Carrega todos os dados de anomalia em um único array enorme 

In [5]:
# Carrega todos os dados de anomalia em um único array
block_size = 5
full_data = []
for rows_index in range(BLOCO_TOTAL):
    # slice data like (n:n+5)
    value = float(file_content[rows_index * block_size: rows_index * block_size + block_size])
    value /= 10
    full_data.append(value)

Para fins de debug

In [34]:
def le_arquivo_mensal(file_name):
    file_content = open(file_name).read()

    file_header = file_content[0:25]

    #Remove header de um único arquivo
    file_content = file_content[25:]
    file_content = file_content.replace("\n", "")

    block_size = 5
    dados_do_arquivo = []
    for rows_index in range(BLOCO_DE_DADOS_POR_MES):
        # slice data like (n:n+5)
        dados_do_arquivo.append(file_content[rows_index * block_size: rows_index * block_size + block_size])

    return dados_do_arquivo




### Adiciona arquivos que nào estão no arquivo geral de anomalias(está horrível, eu sei, mas vou melhorar)


In [39]:
import os
novo_mes = le_arquivo_mensal("funceme_db/anomalia/individual/_Dados_TSMvento_2014_05_sst_total_05_2014_ano.22")
for item in novo_mes:
    full_data.append(item)
    
arquivos_adicionais = []
arquivos_do_diretorio = os.listdir("funceme_db/anomalia/individual")
for arquivo in arquivos_do_diretorio:
    if arquivo.endswith(".22"):
        arquivos_adicionais.append(arquivo)
print(arquivos_adicionais[0])

_Dados_TSMvento_2014_05_sst_total_05_2014_ano.22


In [20]:
meses = []
for i in range(0, BLOCO_TOTAL + BLOCO_DE_DADOS_POR_MES, 950):
#     print(i, i+BLOCO_DE_DADOS_POR_MES)
    mes = full_data[i:i+BLOCO_DE_DADOS_POR_MES]
    meses.append(mes)
print(len(meses))


605


# Criando Pandas a partir dos dados de anomalida da FUNCEME

In [21]:
funceme_df = pd.DataFrame()

In [22]:
columns = range(0,950)
for mes in meses:
    data = np.array(mes)
    row_df = pd.DataFrame(data.reshape(-1, len(data)), columns=colunas_do_data_frame)
    funceme_df = funceme_df.append(row_df)

In [23]:
funceme_df.index = range(0, 605, 1)
funceme_df.shape


(605, 950)

In [24]:
funceme_df.tail()

,60W-30N,60W-28N,60W-26N,60W-24N,60W-22N,60W-20N,60W-18N,60W-16N,60W-14N,60W-12N,...,14E-0S,14E-2S,14E-4S,14E-6S,14E-8S,14E-10S,14E-12S,14E-14S,14E-16S,14E-18S
600,1.6,1.6,1.5,1.4,1.3,1.2,1,0.8,0.7,0.6,...,0.2,0.3,0.4,0.6,0.6,0.5,0.4,0.4,0.4,9999.8
601,1.9,1.7,1.6,1.4,1.2,1,0.8,0.7,0.6,0.5,...,0.8,0.6,0.4,0.2,0,0,0,0.1,0.2,9999.8
602,1.7,1.6,1.4,1.3,1.2,1.2,1.2,1.1,1.1,0.9,...,1,1.1,1.1,1.2,1,0.7,0.5,0.3,0.1,9999.8
603,1.2,1.2,1.1,1,0.9,0.8,0.7,0.6,0.6,0.5,...,0.6,0.7,0.7,0.5,0.2,-0.3,-0.8,-1.2,-1.7,9999.8
604,"[ 6, 6, 5, 5, 6, 5, ...","[ 5, 5, 5, 4, 4, 4, ...","[ 5, 4, 4, 3, 3, 3, ...","[ 5, 4, 3, 3, 2, 2, ...","[ 3, 3, 2, 2, 1, 1, ...","[ 1, 1, 1, 1, 0, -1, ...","[ 0, 0, 0, 0, -1, -2, ...","[ 0, 0, -1, -1, -1, -1, ...","[ 0, 0, -1, -1, -1, -1, ...","[ 1, 0, 0, -1, -1, -1, ...",...,99998,99998,99998,99998,99998,99998,99998,3,3,4


### Setando indices baseados na data


In [25]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

FORMAT = "%Y-%m"
some_date_time1 = "1964-01"
data_inicial = datetime.strptime(some_date_time1, FORMAT)
data_depois = data_inicial + relativedelta(months=1)

In [26]:
indexes_data = []
for i in range(len(meses)):
    indexes_data.append(data_inicial + relativedelta(months=i))

In [27]:
funceme_df=funceme_df.set_index(pd.DatetimeIndex(data=indexes_data))

In [28]:
funceme_df.head()

,60W-30N,60W-28N,60W-26N,60W-24N,60W-22N,60W-20N,60W-18N,60W-16N,60W-14N,60W-12N,...,14E-0S,14E-2S,14E-4S,14E-6S,14E-8S,14E-10S,14E-12S,14E-14S,14E-16S,14E-18S
1964-01-01,0.4,0.4,0.5,0.5,0.5,0.5,0.4,0.2,0,0,...,0.4,0.7,0.9,1.1,0.8,0.9,1.1,1.3,1.8,9999.8
1964-02-01,0.2,0,0,-0.2,-0.2,-0.2,0.1,0.3,0.4,0.7,...,-0.2,-0.1,0,-0.2,-0.3,-0.2,-0.1,0,0.2,9999.8
1964-03-01,-0.1,0.1,0.2,0.3,0.4,0.4,0.5,0.5,0.5,0.5,...,-0.5,-0.6,-0.6,-0.5,-0.4,-0.2,0,0.1,0.3,9999.8
1964-04-01,0.8,0.6,0.5,0.6,0.5,0.5,0.5,0.5,0.5,0.5,...,0.2,-0.4,-0.7,-0.8,-0.8,-0.1,0.4,1,1.5,9999.8
1964-05-01,0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.5,0.4,...,0.2,0.1,-0.1,-0.6,-1,-1.3,-1.7,-2,-2.2,9999.8


In [29]:
funceme_df.tail()

,60W-30N,60W-28N,60W-26N,60W-24N,60W-22N,60W-20N,60W-18N,60W-16N,60W-14N,60W-12N,...,14E-0S,14E-2S,14E-4S,14E-6S,14E-8S,14E-10S,14E-12S,14E-14S,14E-16S,14E-18S
2014-01-01,1.6,1.6,1.5,1.4,1.3,1.2,1,0.8,0.7,0.6,...,0.2,0.3,0.4,0.6,0.6,0.5,0.4,0.4,0.4,9999.8
2014-02-01,1.9,1.7,1.6,1.4,1.2,1,0.8,0.7,0.6,0.5,...,0.8,0.6,0.4,0.2,0,0,0,0.1,0.2,9999.8
2014-03-01,1.7,1.6,1.4,1.3,1.2,1.2,1.2,1.1,1.1,0.9,...,1,1.1,1.1,1.2,1,0.7,0.5,0.3,0.1,9999.8
2014-04-01,1.2,1.2,1.1,1,0.9,0.8,0.7,0.6,0.6,0.5,...,0.6,0.7,0.7,0.5,0.2,-0.3,-0.8,-1.2,-1.7,9999.8
2014-05-01,"[ 6, 6, 5, 5, 6, 5, ...","[ 5, 5, 5, 4, 4, 4, ...","[ 5, 4, 4, 3, 3, 3, ...","[ 5, 4, 3, 3, 2, 2, ...","[ 3, 3, 2, 2, 1, 1, ...","[ 1, 1, 1, 1, 0, -1, ...","[ 0, 0, 0, 0, -1, -2, ...","[ 0, 0, -1, -1, -1, -1, ...","[ 0, 0, -1, -1, -1, -1, ...","[ 1, 0, 0, -1, -1, -1, ...",...,99998,99998,99998,99998,99998,99998,99998,3,3,4


# Decomposição temporal da série


### Adicional imports


In [ ]:
from statsmodels.tsa import stattools
from statsmodels.tsa import seasonal
%matplotlib inline
from matplotlib import pyplot as plt

### Filtro de data

In [ ]:
funceme_df = funceme_df.loc['2009-01-01':'2014-12-01']

# Função que decompões função baseado na lat, long

In [ ]:
def monta_grafico_de_decomposicao(coluna):
    decompose_model = seasonal.seasonal_decompose(funceme_df[coluna].tolist(), freq=12,
                                          model='additive')
    fig, axarr = plt.subplots(4, sharex=True)
    fig.set_size_inches(5.5, 5.5)

    funceme_df[coluna].plot(ax=axarr[0], color='b', linestyle='-')
    axarr[0].set_title('Gráfico mensal')

    pd.Series(data=decompose_model.trend, index=funceme_df.index).plot(color='r', linestyle='-', ax=axarr[1])
    axarr[1].set_title('Gráfico de Tendência')

    pd.Series(data=decompose_model.seasonal, index=funceme_df.index).plot(color='g', linestyle='-', ax=axarr[2])
    axarr[2].set_title('Componente Seasonal')

    pd.Series(data=decompose_model.resid, index=funceme_df.index).plot(color='k', linestyle='-', ax=axarr[3])
    axarr[3].set_title('Variações irregulares')

    plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=2.0)
    plt.xticks(rotation=10)


In [ ]:
monta_grafico_de_decomposicao('60W-30N')

In [ ]:
monta_grafico_de_decomposicao('60W-28N')

In [ ]:
monta_grafico_de_decomposicao('60W-26N')

In [ ]:
monta_grafico_de_decomposicao('60W-10N')

In [ ]:
monta_grafico_de_decomposicao('34W-16S')